## Data to analyze

* FO win % by player over time
* FO on ice % by player over time (% of faceoffs taken by a player when he's on the ice)
* Ref crews that throw more players out
* FO taken in D-zone, O-zone, neutral zone

In [1]:
import requests
import pandas as pd

from dateutil.parser import parse
from pytz import timezone

pd.options.display.max_columns = None

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    display(x)
    pd.reset_option('display.max_rows')


More more helpful information on navigating the NHL Hockey API, see:

https://gitlab.com/dword4/nhlapi/tree/master/

## Play-by-Play Data

`GET https://statsapi.web.nhl.com/api/v1/game/ID/feed/live Returns` all data about a specified game id including play data with on-ice coordinates and post-game details like first, second and third stars and any details about shootouts. The data returned is simply too large at often over 30k lines and is best explored with a JSON viewer.

### Game ID Dictionary
* First 4 digits signify the season start year, ex: `2018` (for the 2018-2019 season)
* Next 2 digits signify the following:
    - 01: Preseason
    - 02: Regular Season
    - 03: Post-Season (Playoffs)
    - 04: All-Star Games
* The final 4 digits signify the game number. Valid range is `0001`-`1271` (until 2020, when the NHL will have 1312 games per season with the addition of Seattle)

Note: for help visualizing JSON see - http://jsonviewer.stack.hu

## Buffalo Sabres @ Carolina Hurricanes 11-Jan 2019

In [2]:
year = '2018'
season = '02'
game_number = '0683'
game_id = year + season + game_number

url = f'https://statsapi.web.nhl.com/api/v1/game/{game_id}/feed/live'
json = requests.get(url).json()

# Create Pandas dataframe for each play from NHL API

In [3]:
# get list of all plays from the game
play_list = json.get('liveData').get('plays').get('allPlays')

# get game date - the date is in UTC format so we need to convert to US - I chose Central time because I'm the boss
get_date_obj = parse("2019-01-12T00:30:00Z")
date_central = get_date_obj.astimezone(timezone('US/Central'))
game_date = date_central.strftime('%Y-%m-%d')

# get game information that does not change by play
game_id = json.get('gameData').get('game').get('pk')
visiting_team_id = json.get('gameData').get('teams').get('away').get('id')
visiting_team_code = json.get('gameData').get('teams').get('away').get('triCode')
home_team_id = json.get('gameData').get('teams').get('home').get('id')
home_team_code = json.get('gameData').get('teams').get('home').get('triCode')

plays = {
    'game_id' : [],
    'game_date' : [],
    'away_team_id' : [],
    'away_team_code' : [],
    'home_team_id' : [],
    'home_team_code' : [],
    'event_id' : [],
    'play_type' : [],
    'play_type_id' : [],
    'play_description' : [],
    'period' : [],
    'time_elapsed' : [],
    'time_remaining' : [],
    'player1_id' : [],
    'player1_name' : [],
    'player2_id' : [],
    'player2_name' : []
}

for play in play_list:
    plays['game_id'].append(game_id)
    plays['game_date'].append(game_date)
    plays["away_team_id"].append(visiting_team_id)
    plays["away_team_code"].append(visiting_team_code)
    plays["home_team_id"].append(home_team_id)
    plays["home_team_code"].append(home_team_code)
    plays['event_id'].append(play.get('about').get('eventId'))
    plays['play_type'].append(play.get('result').get('event'))
    plays['play_type_id'].append(play.get('result').get('eventTypeId'))
    plays['play_description'].append(play.get('result').get('description'))
    plays['period'].append(play.get('about').get('period'))
    plays['time_elapsed'].append(play.get('about').get('periodTime'))
    plays['time_remaining'].append(play.get('about').get('periodTimeRemaining'))
    
    if play.get('players') is None:
        # add None type to the list to make all the lists the same length for the Pandas conversion to DataFrame
        plays['player1_id'].append(None)
        plays['player1_name'].append(None)
        plays['player2_id'].append(None)
        plays['player2_name'].append(None)
    else:
        # add Player 1 info
        player1 = play.get('players')[0]
        plays['player1_id'].append(player1.get('player').get('id'))
        plays['player1_name'].append(player1.get('player').get('fullName'))
        
        # check if there is a second player in the play, if not add None type to the list
        if len(play.get('players')) == 2:
            player2 = play.get('players')[1]
            plays['player2_id'].append(player2.get('player').get('id'))
            plays['player2_name'].append(player2.get('player').get('fullName'))            
        else:
            plays['player2_id'].append(None)
            plays['player2_name'].append(None)        
    
plays_api = pd.DataFrame(plays)
print_full(plays_api)

,game_id,game_date,away_team_id,away_team_code,home_team_id,home_team_code,event_id,play_type,play_type_id,play_description,period,time_elapsed,time_remaining,player1_id,player1_name,player2_id,player2_name
0,2018020683,2019-01-11,7,BUF,12,CAR,1,Game Scheduled,GAME_SCHEDULED,Game Scheduled,1,00:00,20:00,NaN,None,NaN,None
1,2018020683,2019-01-11,7,BUF,12,CAR,4,Period Ready,PERIOD_READY,Period Ready,1,00:00,20:00,NaN,None,NaN,None
2,2018020683,2019-01-11,7,BUF,12,CAR,52,Period Start,PERIOD_START,Period Start,1,00:00,20:00,NaN,None,NaN,None
3,2018020683,2019-01-11,7,BUF,12,CAR,53,Faceoff,FACEOFF,Jack Eichel faceoff won against Sebastian Aho,1,00:00,20:00,8478403.0,Jack Eichel,8478427.0,Sebastian Aho
4,2018020683,2019-01-11,7,BUF,12,CAR,54,Giveaway,GIVEAWAY,Giveaway by Teuvo Teravainen,1,00:22,19:38,8476882.0,Teuvo Teravainen,NaN,None
5,2018020683,2019-01-11,7,BUF,12,CAR,55,Takeaway,TAKEAWAY,Takeaway by Sebastian Aho,1,00:29,19:31,8478427.0,Sebastian Aho,NaN,None
6,2018020683,2019-01-11,7,BUF,12,CAR,56,Hit,HIT,Justin Faulk hit Conor Sheary,1,01:17,18:43,8475753.0,Justin Faulk,8477839.0,Conor Sheary
7,2018020683,2019-01-11,7,BUF,12,CAR,57,Giveaway,GIVEAWAY,Giveaway by Calvin de Haan,1,01:32,18:28,8475177.0,Calvin de Haan,NaN,None
8,2018020683,2019-01-11,7,BUF,12,CAR,7,Shot,SHOT,Dougie Hamilton Wrist Shot saved by Carter Hutton,1,02:08,17:52,8476462.0,Dougie Hamilton,8475622.0,Carter Hutton
9,2018020683,2019-01-11,7,BUF,12,CAR,12,Shot,SHOT,Justin Williams Backhand saved by Carter Hutton,1,02:12,17:48,8468508.0,Justin Williams,8475622.0,Carter Hutton


# Next, scrape the other players on the ice during each event & transfer the data into a Pandas dataframe

In [4]:
import re
from bs4 import BeautifulSoup

In [5]:
# get game id & year codes from the same source as the data above
year_id_scrape = year + str(int(year)+1)
game_id_scrape = season + game_number

url = f'http://www.nhl.com/scores/htmlreports/{year_id_scrape}/PL{game_id_scrape}.HTM'
response = requests.get(url)

# lxml parser required
soup = BeautifulSoup(response.text, "lxml")

In [6]:
# trs - table rows in an HTML table
trs = soup.find_all('tr', class_='evenColor')

In [7]:
# set up dictionary to hold data lists before converting to Pandas dataframe
play_by_play_data = {
    'event_id' : [],
    'period' : [],
    'strength' : [],
    'time_elapsed' : [],
    'time_remaining' : [],
    'play_type' : [],
    'play_description' : [],
    'away_on_ice' : [],
    'home_on_ice' : []
}

# create keys for HTML table data locations
key_lookup = {
    1 : 'event_id',
    3 : 'period',
    5 : 'strength',
    7 : 'time_elapsed',
    8 : 'time_remaining',
    9 : 'play_type',
    11 : 'play_description',
    13 : 'away_on_ice',
    15 : 'home_on_ice'
}

# create lookup table for plays to match the NHL API data
play_lookup = {
    'FAC' : 'FACEOFF',
    'GIVE' : 'GIVEAWAY',
    'TAKE' : 'TAKEAWAY',
    'HIT' : 'HIT',
    'SHOT' : 'SHOT',
    'MISS' : 'MISSED_SHOT',
    'STOP' : 'STOP',
    'BLOCK' : 'BLOCKED_SHOT',
    'GOAL' : 'GOAL',
    'PENL' : 'PENALTY',
    'PEND' : 'PERIOD_END',
    'PSTR' : 'PERIOD_START',
    'GEND' : 'GAME_END'
}

# scrape data from HTML table
pattern = r'\n+'
for row in trs[4:]:
    for i, cell in enumerate(row):     
        if i in [1, 3, 5, 9, 11, 13, 15]:
            text = cell.text.replace('\xa0', ' ')
            text = re.sub(pattern, '', text)
            
            key = key_lookup.get(i)
            play_by_play_data[key].append(text)

# scrape time elapsed data from HTML table separately
# we need to do this because the time elapsed & time remaining columns get merged into one during the scrape
for row in trs[4:]:
    for i, cell in enumerate(row):     
        if i in [7]:
            text = cell.text.replace('\xa0', ' ')
            text = re.sub(pattern, '', text)
            
            delimiter = cell.text.find(':') + 3
            text = cell.text[0:delimiter].zfill(5)
            
            key = key_lookup.get(i)
            play_by_play_data[key].append(text)
            
# scrape time remaining data from HTML table separately
# we need to do this because the time elapsed & time remaining columns get merged into one during the scrape
for row in trs[4:]:
    for i, cell in enumerate(row):     
        if i in [7]:
            text = cell.text.replace('\xa0', ' ')
            text = re.sub(pattern, '', text)
            
            delimiter = cell.text.find(':') + 3
            text = cell.text[delimiter:].zfill(5)
            
            key = key_lookup.get(i + 1)
            play_by_play_data[key].append(text)

# convert data to Pandas dataframe
plays_scrape = pd.DataFrame(play_by_play_data)

# split home on ice column into individual columns
home_on_ice = ( plays_scrape['home_on_ice']
                .str.strip()
                .str.replace('[aA-zZ]', '')
                .str.split(' ', expand=True) )
home_cols = ['home_1', 'home_2', 'home_3', 'home_4', 'home_5', 'home_6']
home_on_ice.columns = home_cols

# split away on ice column into individual columns
away_on_ice = ( plays_scrape['away_on_ice']
                   .str.strip()
                   .str.replace('[aA-zZ]', '')
                   .str.split(' ', expand=True) )
away_cols = ['away_1', 'away_2', 'away_3', 'away_4', 'away_5', 'away_6']
away_on_ice.columns = away_cols

# add the on ice player columns
plays_scrape[away_cols] = away_on_ice
plays_scrape[home_cols] = home_on_ice

# add game id to the df
plays_scrape["game_id"] = game_id

plays_scrape['play_type_id'] = plays_scrape['play_type'].map(play_lookup)

# reorder columns
cols = [ 'game_id',
         'event_id',
         'period',
         'strength',
         'time_elapsed',
         'time_remaining',
         'play_type',
         'play_type_id',
         'play_description',
         'away_on_ice',
         'home_on_ice',
         'away_1','away_2','away_3','away_4','away_5','away_6',
         'home_1','home_2','home_3','home_4','home_5','home_6']
plays_scrape = plays_scrape[cols]
plays_scrape["period"] = pd.to_numeric(plays_scrape["period"])
plays_scrape.head(2)

,game_id,event_id,period,strength,time_elapsed,time_remaining,play_type,play_type_id,play_description,away_on_ice,home_on_ice,away_1,away_2,away_3,away_4,away_5,away_6,home_1,home_2,home_3,home_4,home_5,home_6
0,2018020683,5,1,EV,00:00,20:00,FAC,FACEOFF,BUF won Neu. Zone - BUF #9 EICHEL vs CAR #20 AHO,9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,9,53,72,6,19,40,20,79,86,22,74,35
1,2018020683,6,1,EV,00:22,19:38,GIVE,GIVEAWAY,"CAR GIVEAWAY - #86 TERAVAINEN, Def. Zone",9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,9,53,72,6,19,40,20,79,86,22,74,35


In [8]:
plays_scrape_unpivot = pd.melt(plays_scrape, 
                                id_vars=[
                                         'game_id',
                                         'event_id',
                                         'period',
                                         'strength',
                                         'time_elapsed',
                                         'time_remaining',
                                         'play_type',
                                         'play_type_id',
                                         'play_description',
                                         'away_on_ice',
                                         'home_on_ice'
                                ], 
                                value_vars=[
                                         'away_1','away_2','away_3','away_4','away_5','away_6',
                                         'home_1','home_2','home_3','home_4','home_5','home_6'
                                ],
                                value_name='jersey_number'
                               )

# strip home/away from 'variable' column to create 'home/away' column for joining roster data
home_away = []
for play in plays_scrape_unpivot['variable']:
    home_away.append(play.split("_")[0])

# add home/away column to plays scrape unpivot data frame
plays_scrape_unpivot['home_away'] = home_away

plays_scrape_unpivot.head()

,game_id,event_id,period,strength,time_elapsed,time_remaining,play_type,play_type_id,play_description,away_on_ice,home_on_ice,variable,jersey_number,home_away
0,2018020683,5,1,EV,00:00,20:00,FAC,FACEOFF,BUF won Neu. Zone - BUF #9 EICHEL vs CAR #20 AHO,9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,away_1,9,away
1,2018020683,6,1,EV,00:22,19:38,GIVE,GIVEAWAY,"CAR GIVEAWAY - #86 TERAVAINEN, Def. Zone",9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,away_1,9,away
2,2018020683,7,1,EV,00:29,19:31,TAKE,TAKEAWAY,"CAR TAKEAWAY - #20 AHO, Def. Zone",9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,away_1,9,away
3,2018020683,8,1,EV,01:17,18:43,HIT,HIT,"CAR #27 FAULK HIT BUF #43 SHEARY, Neu. Zone",17C 23C 43L 24D 55D 40G,71C 37R 48L 27D 44D 35G,away_1,17,away
4,2018020683,9,1,EV,01:32,18:28,GIVE,GIVEAWAY,"CAR GIVEAWAY - #44 DE HAAN, Neu. Zone",71C 21R 22L 4D 26D 40G,42C 14R 23L 44D 57D 35G,away_1,71,away


# Get player roster data by game

In [11]:
teams_url = f'https://statsapi.web.nhl.com/api/v1/game/{game_id}/boxscore'
teams_json = requests.get(teams_url).json()
teams_list = teams_json.get('teams')

In [12]:
players = {
    'home_away' : [],
    'team_id' : [],
    'player_id' : [],
    'player_name' : [],
    'player_position' : [],
    'jersey_number' : []
}

for team in teams_list:
    for player in teams_list.get(team).get('players'):
        players['home_away'].append(team)
        players['team_id'].append(teams_list.get(team).get('team').get('id'))
        players['player_id'].append(player)
        players['player_name'].append(teams_list.get(team).get('players').get(player).get('person').get('fullName'))
        players['player_position'].append(teams_list.get(team).get('players').get(player).get('position').get('abbreviation'))
        players['jersey_number'].append(teams_list.get(team).get('players').get(player).get('jerseyNumber'))
    
players = pd.DataFrame(players)
players.head()

,home_away,team_id,player_id,player_name,player_position,jersey_number
0,away,7,ID8469506,Jason Pominville,RW,29
1,away,7,ID8475728,Johan Larsson,C,22
2,away,7,ID8474618,Marco Scandella,D,6
3,away,7,ID8477839,Conor Sheary,LW,43
4,away,7,ID8471436,Matt Hunwick,N/A,48


# Now we will merge the two Pandas dataframes together

In [13]:
plays_merge = plays_api.merge(plays_scrape[
                             ['game_id',
                              'period',
                              'play_type_id',
                              'time_elapsed',
                              'strength',
                              'away_on_ice',
                              'home_on_ice',
                              'away_1','away_2','away_3','away_4','away_5','away_6',
                              'home_1','home_2','home_3','home_4','home_5','home_6'
                             ]
                            ], how = 'inner', on = ['game_id', 'period', 'play_type_id', 'time_elapsed'])

plays_merge.head(100)

,game_id,game_date,away_team_id,away_team_code,home_team_id,home_team_code,event_id,play_type,play_type_id,play_description,period,time_elapsed,time_remaining,player1_id,player1_name,player2_id,player2_name,strength,away_on_ice,home_on_ice,away_1,away_2,away_3,away_4,away_5,away_6,home_1,home_2,home_3,home_4,home_5,home_6
0,2018020683,2019-01-11,7,BUF,12,CAR,53,Faceoff,FACEOFF,Jack Eichel faceoff won against Sebastian Aho,1,00:00,20:00,8478403.0,Jack Eichel,8478427.0,Sebastian Aho,EV,9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,9,53,72,6,19,40,20,79,86,22,74,35
1,2018020683,2019-01-11,7,BUF,12,CAR,54,Giveaway,GIVEAWAY,Giveaway by Teuvo Teravainen,1,00:22,19:38,8476882.0,Teuvo Teravainen,NaN,None,EV,9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,9,53,72,6,19,40,20,79,86,22,74,35
2,2018020683,2019-01-11,7,BUF,12,CAR,55,Takeaway,TAKEAWAY,Takeaway by Sebastian Aho,1,00:29,19:31,8478427.0,Sebastian Aho,NaN,None,EV,9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,9,53,72,6,19,40,20,79,86,22,74,35
3,2018020683,2019-01-11,7,BUF,12,CAR,56,Hit,HIT,Justin Faulk hit Conor Sheary,1,01:17,18:43,8475753.0,Justin Faulk,8477839.0,Conor Sheary,EV,17C 23C 43L 24D 55D 40G,71C 37R 48L 27D 44D 35G,17,23,43,24,55,40,71,37,48,27,44,35
4,2018020683,2019-01-11,7,BUF,12,CAR,57,Giveaway,GIVEAWAY,Giveaway by Calvin de Haan,1,01:32,18:28,8475177.0,Calvin de Haan,NaN,None,EV,71C 21R 22L 4D 26D 40G,42C 14R 23L 44D 57D 35G,71,21,22,4,26,40,42,14,23,44,57,35
5,2018020683,2019-01-11,7,BUF,12,CAR,7,Shot,SHOT,Dougie Hamilton Wrist Shot saved by Carter Hutton,1,02:08,17:52,8476462.0,Dougie Hamilton,8475622.0,Carter Hutton,EV,71C 21R 22L 4D 26D 40G,42C 14R 23L 19D 57D 35G,71,21,22,4,26,40,42,14,23,19,57,35
6,2018020683,2019-01-11,7,BUF,12,CAR,12,Shot,SHOT,Justin Williams Backhand saved by Carter Hutton,1,02:12,17:48,8468508.0,Justin Williams,8475622.0,Carter Hutton,EV,71C 21R 22L 4D 26D 40G,42C 14R 23L 19D 57D 35G,71,21,22,4,26,40,42,14,23,19,57,35
7,2018020683,2019-01-11,7,BUF,12,CAR,14,Missed Shot,MISSED_SHOT,Justin Williams - Wide of Net,1,02:13,17:47,8468508.0,Justin Williams,NaN,None,EV,71C 21R 22L 4D 26D 40G,42C 14R 23L 19D 57D 35G,71,21,22,4,26,40,42,14,23,19,57,35
8,2018020683,2019-01-11,7,BUF,12,CAR,8,Shot,SHOT,Greg McKegg Backhand saved by Carter Hutton,1,02:18,17:42,8475735.0,Greg McKegg,8475622.0,Carter Hutton,EV,71C 21R 22L 4D 26D 40G,42C 14R 23L 19D 57D 35G,71,21,22,4,26,40,42,14,23,19,57,35
9,2018020683,2019-01-11,7,BUF,12,CAR,58,Hit,HIT,Jaccob Slavin hit Jason Pominville,1,02:29,17:31,8476958.0,Jaccob Slavin,8469506.0,Jason Pominville,EV,28C 37C 29R 4D 26D 40G,49C 14R 13L 22D 74D 35G,28,37,29,4,26,40,49,14,13,22,74,35


and voila!!

In [22]:
player_num = 1
plays_merge_on_ice = plays_merge
while player_num <= 6:
    plays_merge_on_ice = plays_merge_on_ice.merge(players[
                               [
                                'team_id',
                                'jersey_number',
                                'player_id',
                                'player_name',
                                'player_position'
                                ]
                               ], how = 'inner', left_on = ['away_team_id', 'away_' + str(player_num)], right_on = ['team_id', 'jersey_number'],
                                  suffixes = ('','_' + str(player_num))
                     )
    plays_merge_on_ice = plays_merge_on_ice.merge(players[
                               [
                                'team_id',
                                'jersey_number',
                                'player_id',
                                'player_name',
                                'player_position'
                                ]
                               ], how = 'inner', left_on = ['home_team_id', 'home_' + str(player_num)], right_on = ['team_id', 'jersey_number'],
                                  suffixes = ('','_' + str(player_num))
                     )
    player_num += 1
plays_merge_on_ice.head(10)


,game_id,game_date,away_team_id,away_team_code,home_team_id,home_team_code,event_id,play_type,play_type_id,play_description,period,time_elapsed,time_remaining,player1_id,player1_name,player2_id,player2_name,strength,away_on_ice,home_on_ice,away_1,away_2,away_3,away_4,away_5,away_6,home_1,home_2,home_3,home_4,home_5,home_6,team_id,jersey_number,player_id,player_name,player_position,team_id_1,jersey_number_1,player_id_1,player_name_1,player_position_1,team_id_2,jersey_number_2,player_id_2,player_name_2,player_position_2,team_id_2,jersey_number_2,player_id_2,player_name_2,player_position_2,team_id_3,jersey_number_3,player_id_3,player_name_3,player_position_3,team_id_3,jersey_number_3,player_id_3,player_name_3,player_position_3,team_id_4,jersey_number_4,player_id_4,player_name_4,player_position_4,team_id_4,jersey_number_4,player_id_4,player_name_4,player_position_4,team_id_5,jersey_number_5,player_id_5,player_name_5,player_position_5,team_id_5,jersey_number_5,player_id_5,player_name_5,player_position_5,team_id_6,jersey_number_6,player_id_6,player_name_6,player_position_6,team_id_6,jersey_number_6,player_id_6,player_name_6,player_position_6
0,2018020683,2019-01-11,7,BUF,12,CAR,53,Faceoff,FACEOFF,Jack Eichel faceoff won against Sebastian Aho,1,00:00,20:00,8478403.0,Jack Eichel,8478427.0,Sebastian Aho,EV,9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,9,53,72,6,19,40,20,79,86,22,74,35,7,9,ID8478403,Jack Eichel,C,12,20,ID8478427,Sebastian Aho,C,7,53,ID8475784,Jeff Skinner,LW,12,79,ID8475907,Micheal Ferland,LW,7,72,ID8479420,Tage Thompson,RW,12,86,ID8476882,Teuvo Teravainen,LW,7,6,ID8474618,Marco Scandella,D,12,22,ID8477488,Brett Pesce,D,7,19,ID8476931,Jake McCabe,D,12,74,ID8476958,Jaccob Slavin,D,7,40,ID8475622,Carter Hutton,G,12,35,ID8470147,Curtis McElhinney,G
1,2018020683,2019-01-11,7,BUF,12,CAR,54,Giveaway,GIVEAWAY,Giveaway by Teuvo Teravainen,1,00:22,19:38,8476882.0,Teuvo Teravainen,NaN,None,EV,9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,9,53,72,6,19,40,20,79,86,22,74,35,7,9,ID8478403,Jack Eichel,C,12,20,ID8478427,Sebastian Aho,C,7,53,ID8475784,Jeff Skinner,LW,12,79,ID8475907,Micheal Ferland,LW,7,72,ID8479420,Tage Thompson,RW,12,86,ID8476882,Teuvo Teravainen,LW,7,6,ID8474618,Marco Scandella,D,12,22,ID8477488,Brett Pesce,D,7,19,ID8476931,Jake McCabe,D,12,74,ID8476958,Jaccob Slavin,D,7,40,ID8475622,Carter Hutton,G,12,35,ID8470147,Curtis McElhinney,G
2,2018020683,2019-01-11,7,BUF,12,CAR,55,Takeaway,TAKEAWAY,Takeaway by Sebastian Aho,1,00:29,19:31,8478427.0,Sebastian Aho,NaN,None,EV,9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,9,53,72,6,19,40,20,79,86,22,74,35,7,9,ID8478403,Jack Eichel,C,12,20,ID8478427,Sebastian Aho,C,7,53,ID8475784,Jeff Skinner,LW,12,79,ID8475907,Micheal Ferland,LW,7,72,ID8479420,Tage Thompson,RW,12,86,ID8476882,Teuvo Teravainen,LW,7,6,ID8474618,Marco Scandella,D,12,22,ID8477488,Brett Pesce,D,7,19,ID8476931,Jake McCabe,D,12,74,ID8476958,Jaccob Slavin,D,7,40,ID8475622,Carter Hutton,G,12,35,ID8470147,Curtis McElhinney,G
3,2018020683,2019-01-11,7,BUF,12,CAR,88,Faceoff,FACEOFF,Jack Eichel faceoff won against Sebastian Aho,1,10:25,09:35,8478403.0,Jack Eichel,8478427.0,Sebastian Aho,EV,9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,9,53,72,6,19,40,20,79,86,22,74,35,7,9,ID8478403,Jack Eichel,C,12,20,ID8478427,Sebastian Aho,C,7,53,ID8475784,Jeff Skinner,LW,12,79,ID8475907,Micheal Ferland,LW,7,72,ID8479420,Tage Thompson,RW,12,86,ID8476882,Teuvo Teravainen,LW,7,6,ID8474618,Marco Scandella,D,12,22,ID8477488,Brett Pesce,D,7,19,ID8476931,Jake McCabe,D,12,74,ID8476958,Jaccob Slavin,D,7,40,ID8475622,Carter Hutton,G,12,35,ID8470147,Curtis McElhinney,G
4,2018020683,2019-01-11,7,BUF,12,CAR,89,Giveaway,GIVEAWAY,Giveaway by Sebastian Aho,1,10:36,09:24,8478427.0,Sebastian Aho,NaN,None,EV,9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,9,53,72,6,19,40,20,79,86,22,74,35,7,9,ID8478403,Jack Eichel,C,12,20,ID8478427,Sebastian Aho,C,7,53,ID8475784,Jeff Skinner,LW,12,79,ID8475907,Micheal Ferland,LW,7,72,ID8479420,Tage Thompson,RW,12,86,ID847688

In [ ]:
'game_id',
 'game_date',
 'away_team_id',
 'away_team_code',
 'home_team_id',
 'home_team_code',
 'event_id',
 'play_type',
 'play_type_id',
 'play_description',
 'period',
 'time_elapsed',
 'time_remaining',
 'player1_id',
 'player1_name',
 'player2_id',
 'player2_name',
 'strength',
 'away_on_ice',
 'home_on_ice',
 'away_1',
 'away_2',
 'away_3',
 'away_4',
 'away_5',
 'away_6',
 'home_1',
 'home_2',
 'home_3',
 'home_4',
 'home_5',
 'home_6'

In [23]:
def get_plays_by_type(play_type, number_of_records = 10):
    plays_by_type = plays_merge_on_ice[plays_merge_on_ice["play_type_id"] == play_type].head(number_of_records)
    return plays_by_type

In [24]:
get_plays_by_type('FACEOFF')

,game_id,game_date,away_team_id,away_team_code,home_team_id,home_team_code,event_id,play_type,play_type_id,play_description,period,time_elapsed,time_remaining,player1_id,player1_name,player2_id,player2_name,strength,away_on_ice,home_on_ice,away_1,away_2,away_3,away_4,away_5,away_6,home_1,home_2,home_3,home_4,home_5,home_6,team_id,jersey_number,player_id,player_name,player_position,team_id_1,jersey_number_1,player_id_1,player_name_1,player_position_1,team_id_2,jersey_number_2,player_id_2,player_name_2,player_position_2,team_id_2,jersey_number_2,player_id_2,player_name_2,player_position_2,team_id_3,jersey_number_3,player_id_3,player_name_3,player_position_3,team_id_3,jersey_number_3,player_id_3,player_name_3,player_position_3,team_id_4,jersey_number_4,player_id_4,player_name_4,player_position_4,team_id_4,jersey_number_4,player_id_4,player_name_4,player_position_4,team_id_5,jersey_number_5,player_id_5,player_name_5,player_position_5,team_id_5,jersey_number_5,player_id_5,player_name_5,player_position_5,team_id_6,jersey_number_6,player_id_6,player_name_6,player_position_6,team_id_6,jersey_number_6,player_id_6,player_name_6,player_position_6
0,2018020683,2019-01-11,7,BUF,12,CAR,53,Faceoff,FACEOFF,Jack Eichel faceoff won against Sebastian Aho,1,00:00,20:00,8478403.0,Jack Eichel,8478427.0,Sebastian Aho,EV,9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,9,53,72,6,19,40,20,79,86,22,74,35,7,9,ID8478403,Jack Eichel,C,12,20,ID8478427,Sebastian Aho,C,7,53,ID8475784,Jeff Skinner,LW,12,79,ID8475907,Micheal Ferland,LW,7,72,ID8479420,Tage Thompson,RW,12,86,ID8476882,Teuvo Teravainen,LW,7,6,ID8474618,Marco Scandella,D,12,22,ID8477488,Brett Pesce,D,7,19,ID8476931,Jake McCabe,D,12,74,ID8476958,Jaccob Slavin,D,7,40,ID8475622,Carter Hutton,G,12,35,ID8470147,Curtis McElhinney,G
3,2018020683,2019-01-11,7,BUF,12,CAR,88,Faceoff,FACEOFF,Jack Eichel faceoff won against Sebastian Aho,1,10:25,09:35,8478403.0,Jack Eichel,8478427.0,Sebastian Aho,EV,9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,9,53,72,6,19,40,20,79,86,22,74,35,7,9,ID8478403,Jack Eichel,C,12,20,ID8478427,Sebastian Aho,C,7,53,ID8475784,Jeff Skinner,LW,12,79,ID8475907,Micheal Ferland,LW,7,72,ID8479420,Tage Thompson,RW,12,86,ID8476882,Teuvo Teravainen,LW,7,6,ID8474618,Marco Scandella,D,12,22,ID8477488,Brett Pesce,D,7,19,ID8476931,Jake McCabe,D,12,74,ID8476958,Jaccob Slavin,D,7,40,ID8475622,Carter Hutton,G,12,35,ID8470147,Curtis McElhinney,G
7,2018020683,2019-01-11,7,BUF,12,CAR,226,Faceoff,FACEOFF,Jack Eichel faceoff won against Sebastian Aho,2,00:00,20:00,8478403.0,Jack Eichel,8478427.0,Sebastian Aho,EV,9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,9,53,72,6,19,40,20,79,86,22,74,35,7,9,ID8478403,Jack Eichel,C,12,20,ID8478427,Sebastian Aho,C,7,53,ID8475784,Jeff Skinner,LW,12,79,ID8475907,Micheal Ferland,LW,7,72,ID8479420,Tage Thompson,RW,12,86,ID8476882,Teuvo Teravainen,LW,7,6,ID8474618,Marco Scandella,D,12,22,ID8477488,Brett Pesce,D,7,19,ID8476931,Jake McCabe,D,12,74,ID8476958,Jaccob Slavin,D,7,40,ID8475622,Carter Hutton,G,12,35,ID8470147,Curtis McElhinney,G
10,2018020683,2019-01-11,7,BUF,12,CAR,334,Faceoff,FACEOFF,Sebastian Aho faceoff won against Jack Eichel,2,08:25,11:35,8478427.0,Sebastian Aho,8478403.0,Jack Eichel,EV,9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,9,53,72,6,19,40,20,79,86,22,74,35,7,9,ID8478403,Jack Eichel,C,12,20,ID8478427,Sebastian Aho,C,7,53,ID8475784,Jeff Skinner,LW,12,79,ID8475907,Micheal Ferland,LW,7,72,ID8479420,Tage Thompson,RW,12,86,ID8476882,Teuvo Teravainen,LW,7,6,ID8474618,Marco Scandella,D,12,22,ID8477488,Brett Pesce,D,7,19,ID8476931,Jake McCabe,D,12,74,ID8476958,Jaccob Slavin,D,7,40,ID8475622,Carter Hutton,G,12,35,ID8470147,Curtis McElhinney,G
14,2018020683,2019-01-11,7,BUF,12,CAR,422,Faceoff,FACEOFF,Sebastian Aho faceoff won against Jack Eichel,2,14:33,05:27,8478427.0,Sebastian Aho,8478403.0,Jack Eichel,EV,9C 53C 29R 6D 19D 40G,20R 79L 86L 22D 74D 35G,9,53,29,6,19,40,20,79,86,22,74,35,7,9,ID8478403,Jack Eichel,C,12,20,ID8478427,Sebastian Aho,C,7,53,ID8475784,Jeff 